In [ ]:
# Librerías
import os
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import joblib
from sklearn.model_selection import cross_val_score


# Cambiar directorio a 'outputs' para leer los CSV
os.chdir("C:/repo personal/PYTHON/deteccion de fraudes/outputs")

# Subir un nivel desde 'modelos/' a la raíz 'deteccion de fraudes/'
sys.path.append(os.path.abspath('..'))

# Importar la función ajustar_umbral desde utils.py
from scripts.utils import ajustar_umbral

In [ ]:
# Cargar datos
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv").squeeze()
y_test = pd.read_csv("y_test.csv").squeeze()

print(f"Tamaño de X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}, y_test: {y_test.shape}")

In [ ]:
# Aplicar SMOTE para balancear los datos de entrenamiento
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("Datos balanceados con SMOTE:")
print(f"Tamaño original: {X_train.shape}, {y_train.shape}")
print(f"Tamaño balanceado: {X_train_balanced.shape}, {y_train_balanced.shape}")

In [ ]:
# Calcular la proporción de clases para penalizaciones de error por fraude
ratio_neg_pos = np.sum(y_train_balanced == 0) / np.sum(y_train_balanced == 1)

# Definir el modelo base
xgb = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='aucpr',
    scale_pos_weight=ratio_neg_pos,
    n_jobs=-1
)

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [200, 300, 400, 500],
    'max_depth': [2, 3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 1, 5],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5, 2]
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,
    scoring='average_precision',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Ajustar el modelo
random_search.fit(X_train_balanced, y_train_balanced)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", random_search.best_params_)

In [ ]:
# Guardar el objeto completo
joblib.dump(random_search, '../outputs/random_search_xgb.pkl')

# Para usar después y no correr el modelo completo de nuevo:
random_search = joblib.load('../outputs/random_search_xgb.pkl')

In [ ]:
# Entrenar el modelo con los mejores hiperparámetros
best_xgb = random_search.best_estimator_
best_xgb.fit(X_train_balanced, y_train_balanced)

# Predicciones en el conjunto de prueba
y_pred = best_xgb.predict(X_test)
y_prob = best_xgb.predict_proba(X_test)[:, 1]

In [ ]:
# Ajustar el umbral de predicción
thresholds = np.arange(0.4, 0.9, 0.02)
resultados = ajustar_umbral(y_test, y_prob, thresholds)

# Convertir resultados a un DataFrame para visualización
resultados_df = pd.DataFrame(resultados, columns=["Threshold", "Precision", "Recall", "F1-Score"])

# Mostrar resultados
print(resultados_df)

In [ ]:
# Visualizar métricas en función del umbral
plt.figure(figsize=(10, 6))
plt.plot(resultados_df["Threshold"], resultados_df["Precision"], label="Precision", marker="o")
plt.plot(resultados_df["Threshold"], resultados_df["Recall"], label="Recall", marker="o")
plt.plot(resultados_df["Threshold"], resultados_df["F1-Score"], label="F1-Score", marker="o")
plt.title("Métricas en función del umbral")
plt.xlabel("Umbral")
plt.ylabel("Valor de la métrica")
plt.legend()
plt.grid()
plt.show()

# Seleccionar el mejor umbral basado en F1-Score
mejor_umbral = resultados_df.loc[resultados_df["F1-Score"].idxmax(), "Threshold"]
print(f"Mejor umbral basado en F1-Score: {mejor_umbral:.2f}")

# Generar predicciones finales con el mejor umbral
y_pred_final = (y_prob >= mejor_umbral).astype(int)

In [ ]:
# Matriz de confusión con el mejor umbral
conf_matrix = confusion_matrix(y_test, y_pred_final)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Clase 0", "Clase 1"], yticklabels=["Clase 0", "Clase 1"])
plt.title(f"Matriz de Confusión (Umbral = {mejor_umbral:.2f})")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

In [ ]:
# Reporte de clasificación con el mejor umbral
print("Reporte de Clasificación con el mejor umbral:")
print(classification_report(y_test, y_pred_final, digits=4))

In [ ]:
# Calcular Precision-Recall AUC
precision, recall, _ = precision_recall_curve(y_test, y_prob)
pr_auc = auc(recall, precision)
print(f"PR-AUC: {pr_auc:.4f}")

# Visualizar curva Precision-Recall
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f"PR-AUC = {pr_auc:.4f}", color="blue")
plt.title("Curva Precision-Recall")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
# Cross Validation Externa para validar robustez

# Evaluar usando 5-fold Cross Validation en el set de entrenamiento
cv_scores = cross_val_score(
    best_xgb, X_train_balanced, y_train_balanced,
    scoring='average_precision',  # PR-AUC promedio
    cv=5,
    n_jobs=-1
)

print("PR-AUC promedio por Cross-Validation:", cv_scores.mean())
print("PR-AUC de cada fold:", cv_scores)
